In [5]:
tr_medikal_prompt = """Siz bir AI Medikal Asistan Chatbot'sunuz ve medikal soruları cevaplamak üzere eğitildiniz. Aşağıda, bir görevi tanımlayan bir talimat ve yanıt bağlamı verilmiştir. Talebe uygun bir yanıt yazın.

### Kullanıcı:
{}


### Asistan:
{}"""

In [2]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-gpt2")

/Users/mab/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import pandas as pd

data = pd.read_parquet("../veri/doktorsitesi/sohbetler167732.parquet")
data = data.dropna()
data = data.drop_duplicates()
data

,doctor_title,doctor_speciality,question_content,question_answer
0,Dt.,dis-hekimi,60 yaşında babamın dişlerini yaptırmak istiyor...,tabı ben patnostayım buyrun yardımcı olmak ıst...
1,Dt.,dis-hekimi,bir aydır ağzımda geçmeyen ufak bir yara var a...,protez travmasına baglı epulıs gelısebılır yad...
2,Dr. Dt.,dis-hekimi,Merhaba dun dis cekimi yapildiktan sonra bu sa...,Hayır.. rahat olun.normal şeyler yazmıssiniz.....
3,Dr. Dt.,dis-hekimi,"Merhaba.1,5 ay önce köprü dişlerim yapistirild...",Öncelikle geçmiş olsun. Köprü yapımından sonra...
4,Dr. Dt.,dis-hekimi,Merhaba doktor bey bnm çocuğumun üst dişlerind...,bazen lekeler çürük izlenimi verebiliyor.muaye...
...,...,...,...,...
167727,Doç. Dr.,psikiyatri,İyi günler ben yaklasık 1 senedir cipralex 15 ...,"Merhaba, Cipralex (etken maddesi essitalopram)..."
167728,Doç. Dr.,psikiyatri,Mrb ben 6.sınıf tıp öğrencisiyim yaklaşık 1.5 ...,"Merhaba Emin, Öncelikle geçmiş olsun. Anlattık..."
167729,Doç. Dr.,psikiyatri,"Merhaba, Bu yazacagim benim icin cok onemli ve...","Değerli danışan, Hastaları muayene etmeden ila..."
167730,Doç. Dr.,psikiyatri,slm hocam benim ablama yenileyen depresif bozu...,Bahsettiğiniz hastalığın yalnızca depresif boz...


In [19]:
text = ""
for i, row in data.iterrows():
    text += tr_medikal_prompt.format(row["question_content"], row["question_answer"])

text[:1000]

"Siz bir AI Medikal Asistan Chatbot'sunuz ve medikal soruları cevaplamak üzere eğitildiniz. Aşağıda, bir görevi tanımlayan bir talimat ve yanıt bağlamı verilmiştir. Talebe uygun bir yanıt yazın.\n\n### Kullanıcı:\n60 yaşında babamın dişlerini yaptırmak istiyoruz. yardımcı olabilir misiniz?\n\n\n### Asistan:\ntabı ben patnostayım buyrun yardımcı olmak ısterımSiz bir AI Medikal Asistan Chatbot'sunuz ve medikal soruları cevaplamak üzere eğitildiniz. Aşağıda, bir görevi tanımlayan bir talimat ve yanıt bağlamı verilmiştir. Talebe uygun bir yanıt yazın.\n\n### Kullanıcı:\nbir aydır ağzımda geçmeyen ufak bir yara var alt çenemde protezdenmi acaba ne yapmalıyım kötü bir hastalık olur diye korkmaya başladım lütfen beni aydınlatırmısınız meliha özkan\n\n\n### Asistan:\nprotez travmasına baglı epulıs gelısebılır yada patolojık bi durum olabılır bulunudugunuz yerde fakulte varsa cene hastalıkları yada cerrahısıne gorunmenız onerılırSiz bir AI Medikal Asistan Chatbot'sunuz ve medikal soruları cevap

In [20]:
tokens = tokenizer.encode(text)
tokens[:20]

[6050,
 307,
 37370,
 38022,
 45538,
 12840,
 31548,
 2602,
 1230,
 299,
 28595,
 8968,
 2394,
 1384,
 1332,
 3569,
 268,
 5973,
 14,
 17849]

In [21]:
import numpy as np

def write_datafile(filename, toks):
    """
    Saves token data as a .bin file, for reading in C.
    - First comes a header with 256 int32s
    - The tokens follow, each as a uint16
    """
    assert len(toks) < 2**31, "token count too large" # ~2.1B tokens
    # construct the header
    header = np.zeros(256, dtype=np.int32)
    header[0] = 20240520 # magic
    header[1] = 1 # version
    header[2] = len(toks) # number of tokens after the 256*4 bytes of header (each 2 bytes as uint16)
    # construct the tokens numpy array, if not already
    if not isinstance(toks, np.ndarray) or not toks.dtype == np.uint16:
        # validate that no token exceeds a uint16
        maxtok = 2**16
        assert all(0 <= t < maxtok for t in toks), "token dictionary too large for uint16"
        toks_np = np.array(toks, dtype=np.uint16)
    else:
        toks_np = toks
    # write to file
    print(f"writing {len(toks):,} tokens to {filename}")
    with open(filename, "wb") as f:
        f.write(header.tobytes())
        f.write(toks_np.tobytes())

In [22]:
import os

# split into train and validation sets % 10% validation set

val_tokens = tokens[:len(tokens)//10]
train_tokens = tokens[len(tokens)//10:]

# save to file
val_filename = os.path.join("", "gpt_val_tokens.bin")
train_filename = os.path.join("", "gpt_train_tokens.bin")
write_datafile(val_filename, val_tokens)
write_datafile(train_filename, train_tokens)

writing 3,247,308 tokens to gpt_val_tokens.bin
writing 29,225,773 tokens to gpt_train_tokens.bin
